# Importing Packages Needed

In [ ]:
import numpy as np

In [ ]:
import scipy as sp

In [ ]:
import matplotlib.pyplot as py

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
from scipy.signal import find_peaks

In [ ]:
import collections as col

# Definition of Named Tuples

In [ ]:
component = col.namedtuple('component','phase intensity')

In [ ]:
ind_sub = col.namedtuple('ind_sub','phase intensity width')

In [ ]:
pulse_sub = col.namedtuple('pulse_sub','frequency sub1 sub2, errors')

In [ ]:
sub_separation = col.namedtuple('sub_separation','frequency separation')

# Definition of Functions

In [ ]:
def chsq(observed_values, expected_values, err):
        test_statistic=0
        for observed, expected in zip(observed_values, expected_values):
            test_statistic+=((float(observed)-float(expected))/float(err))**2
        return test_statistic

In [ ]:
def sigmaClip(data, alpha=3, tol=0.1, ntrials=10):
    """
    Sigma clipping operation:
    Compute the data's median, m, and its standard deviation, sigma.
    Keep only the data that falls in the range (m-alpha*sigma,m+alpha*sigma) for some value of alpha, and discard everything else.

    This operation is repeated ntrials number of times or until the tolerance level is hit.

    Parameters:
    -----------
    data: list
        A list of floats - the data to clip
    alpha: float
        OPTIONAL - Determines the number of sigmas to use to determine the upper nad lower limits. Default=3
    tol: float
        OPTIONAL - The fractional change in the standard deviation that determines when the tolerance is hit. Default=0.1
    ntrils: int
        OPTIONAL - The maximum number of times to apply the operation. Default=10

    Returns:
    --------
    oldstd: float
        The std of the clipped data
    x: list
        The data list that contains only noise, with nans in place of 'real' data
    """
    x = np.copy(data)
    oldstd = np.nanstd(x)
    #When the x[x<lolim] and x[x>hilim] commands encounter a nan it produces a
    #warning. This is expected because it is ignoring flagged data from a
    #previous trial so the warning is supressed.
    old_settings = np.seterr(all='ignore')
    for trial in range(ntrials):
        median = np.nanmedian(x)

        lolim = median - alpha * oldstd
        hilim = median + alpha * oldstd
        x[x<lolim] = np.nan
        x[x>hilim] = np.nan

        newstd = np.nanstd(x)
        tollvl = (oldstd - newstd) / newstd

        if tollvl <= tol:
            print("Took {0} trials to reach tolerance".format(trial+1))
            np.seterr(**old_settings)
            return oldstd, x

        if trial + 1 == ntrials:
            print("Reached number of trials without reaching tolerance level")
            np.seterr(**old_settings)
            return oldstd, x

        oldstd = newstd

Gaussian Fitting Functions:

In [ ]:
def gauss(x,cen,sd,amp):
    return amp*np.exp(-(x-cen)**2/2/sd**2)

def twogauss(x,cen1,sd1,amp1,cen2,sd2,amp2):
    return gauss(x,cen1,sd1,amp1)+gauss(x,cen2,sd2,amp2)

def threegauss(x,cen1,sd1,amp1,cen2,sd2,amp2,cen3,sd3,amp3):
    return gauss(x,cen1,sd1,amp1)+gauss(x,cen2,sd2,amp2)+gauss(x,cen3,sd3,amp3)

In [ ]:
def single_fit(x, y, error, expect):
    params,cov=curve_fit(gauss,x,y,expect,bounds=([-np.inf,-np.inf,0],[np.inf,np.inf,np.inf]))
    fit=gauss(x,*params)
    
    chi=chsq(y, fit, error)
    chir=chi/(len(y)-1)
    diff=np.sqrt((chir-1)**2)
        
    return params, cov, fit, chi

def double_fit(x, y, error, expect):
    params,cov=curve_fit(twogauss,x,y,expect,bounds=([-np.inf,-np.inf,0,-np.inf,-np.inf,0],[np.inf,np.inf,np.inf,np.inf,np.inf,np.inf]))
    fit=twogauss(x,*params)

    chi=chsq(y, fit, error)
    chir=chi/(len(y)-1)
    diff=np.sqrt((chir-1)**2)
    
    return params, cov, fit, chi

def triple_fit(x, y, error, expect):
    params,cov=curve_fit(threegauss,x,y,expect,bounds=([-np.inf,-np.inf,0,-np.inf,-np.inf,0,-np.inf,-np.inf,0],[np.inf,np.inf,np.inf,np.inf,np.inf,np.inf,np.inf,np.inf,np.inf]))
    fit=threegauss(x,*params)
    
    chi=chsq(y, fit, error)
    chir=chi/(len(y)-1)
    diff=np.sqrt((chir-1)**2)
    
    return params, cov, fit, chi

In [ ]:
def finding_single_fit(x,y, error):
    
    status=100
    
    expect=[500,75,50]
    try:
        params1, cov1, fit1, chi1 = single_fit(x, y, error, expect)
    except:
        chi1=np.inf
        diff1=np.inf
    
    expect.extend((750,75,50))
    try:
        params2, cov2, fit2, chi2 = double_fit(x, y, error, expect)
    except:
        chi2=np.inf
        diff2=np.inf

    if chi1<=chi2 and chi1!=np.inf: 
        status=1

    elif chi2<=chi1 and chi2!=np.inf:
        status=2
        
    if chi1==np.inf and chi2==np.inf: 
        status=0
    
    return status

In [ ]:
def find_fit(no_freq_bands,averageprofiles):
    count0=0
    count1=0
    count2=0

    for i in range(0,no_freq_bands):
        freqband = i
  
        f = averageprofiles['frequency'] == freqband

        x=averageprofiles['phase'][f]
        y=averageprofiles['intensity'][f]
    
        status=finding_single_fit(x,y,error)

        if status==1:
            count1=count1+1
        elif status==2:
            count2=count2+1
        elif status==0:
            count0=count0+1

    if count0>=count1 and count0>=count2:
        fitting=0
    elif count1>=count2 and count1>=count0:
        fitting=1
    elif count2>=count1 and count2>=count0:
        fitting=2
        
    return fitting

In [ ]:
def fitting_gaussians(x,y, error, fitting):
    py.plot(x,y,lw=0.1)
    
    
    if fitting==1:
        expect=[500,75,50]
        try:
            params, cov, fit, chi = single_fit(x, y, error, expect)
            sigma=np.sqrt(np.diag(cov))
            py.plot(x,fit, color='red')
        except:
            params=0
            cov=0
            fit=[]
            chi=np.inf
            sigma=0
    
    elif fitting==2:
        expect=[500,75,50,750,75,50]
        try:
            params, cov, fit, chi = double_fit(x, y, error, expect)
            sigma=np.sqrt(np.diag(cov))
            py.plot(x,fit, color='green')
        except:
            params=0
            cov=0
            fit=[]
            chi=np.inf
            sigma=0
            
    if chi>1000:
        params=0
        cov=0
        fit=[]
        chi=np.inf
        sigma=0
        
    return fit, params, sigma, chi

Finding Peak of Fittings Function

In [ ]:
def gauss_peaks(fit, nofreqband, freqband, sigma, fitting):
    print('Fitting')
    print(fitting)
    
    status='starting'
  
    freq=(((500-300)/nofreqband)*(nofreqband-freqband))+300

    peakpos = find_peaks(fit, prominence=0.2)[0]

    
    if fitting==2:
        if len(peakpos) == 2:
            status='pending'

            if peakpos[0]>=200 and peakpos[0]<=600 and peakpos[1]>=600 and peakpos[1]<=900:
                status='complete'
            else:
                status='error'
        else:
            status='error'

        if status=='complete':
            component1 = component(phase=(360/1024)*peakpos[0], intensity=fit[peakpos[0]])
            component2 = component(phase=(360/1024)*peakpos[1], intensity=fit[peakpos[1]])
            errors = sigma
  
        elif status=='error':
            component1 = component(phase=0, intensity=0)
            component2 = component(phase=0, intensity=0)
            errors = 0
        
        
    elif fitting==1:
        if len(peakpos) == 1:
            status='pending'

            if (peakpos[0]>=200 and peakpos[0]<=600) or (peakpos[0]>=600 and peakpos[0]<=900):
                status='complete'
            else:
                status='error'
        else:
            status='error'

        if status=='complete':
            component1 = component(phase=(360/1024)*peakpos[0], intensity=fit[peakpos[0]])
            component2 = component(phase=0, intensity=0)
            errors = sigma
  
        elif status=='error':
            component1 = component(phase=0, intensity=0)
            component2 = component(phase=0, intensity=0)
            errors = 0
        
    else:
        print("Error: Something not complete in peak finding")

        
    pulse_subp = pulse_sub(frequency=freq, sub1=component1, sub2=component2, errors=sigma)
        
    return pulse_subp

In [ ]:
def ind_peaks(params, nofreqband, freqband, sigma):
  status='starting'
  uncert=[]

  if params[0]<params[3] and params[0]<params[6] and params[0]>=300 and params[0]<=535:
    component1 = ind_component(phase=(360/1024)*params[0], intensity=params[1], width=params[2])
    uncert.extend((sigma[0], sigma[1], sigma[2]))
    status='complete'
  elif params[3]<params[0] and params[3]<params[6] and params[3]>=300 and params[3]<=535:
    component1 = ind_component(phase=(360/1024)*params[3], intensity=params[4], width=params[5])
    uncert.extend((sigma[3], sigma[4], sigma[5]))
    status='complete'
  elif params[6]<params[0] and params[6]<params[3] and params[6]>=300 and params[6]<=535:
    component1 = ind_component(phase=(360/1024)*params[6], intensity=params[7], width=params[8])
    uncert.extend((sigma[6], sigma[7], sigma[8]))
    status='complete'
  else:
    component1 = ind_component(phase=0, intensity=0, width=0)
    uncert.extend((0,0,0))
    status='error'

  if params[0]>params[3] and params[0]>params[6] and params[0]>=650 and params[0]<=800:
    component2 = ind_component(phase=(360/1024)*params[0], intensity=params[1], width=params[2])
    uncert.extend((sigma[0], sigma[1], sigma[2]))
    status='complete'
  elif params[3]>params[0] and params[3]>params[6] and params[3]>=650 and params[3]<=800:
    component2 = ind_component(phase=(360/1024)*params[3], intensity=params[4], width=params[5])
    uncert.extend((sigma[3], sigma[4], sigma[5]))
    status='complete'
  elif params[6]>params[0] and params[6]>params[3] and params[6]>=650 and params[6]<=800:
    component2 = ind_component(phase=(360/1024)*params[6], intensity=params[7], width=params[8])
    uncert.extend((sigma[6], sigma[7], sigma[8]))
    status='complete'
  else:
    component2 = ind_component(phase=0, intensity=0, width=0)
    uncert.extend((0,0,0))
    status='error'

  freq=(((500-300)/nofreqband)*(nofreqband-freqband))+300 

  profile_comp = pulse_component(frequency=freq, comp1=component1, comp2=component2, errors=uncert)

  #Can be used for error checking
  #if status != 'error':
    #plot_peaks(x,y,profile_comp)

  return profile_comp

Plotting Fittings Functions

In [ ]:
def plot_fit(x,y,fit,params,sigma):
    py.plot(x, y, lw=0.5, label='data')
    py.plot(x,fit, color='red',lw=3,label='gaussian fit')
    py.xlabel('Phase Bin')
    py.ylabel('Intensity')
    py.legend()
    return

In [ ]:
def plot_fill(x,y,params):
    params1 = params[0:3]
    params2 = params[3:6]
    gfit1 = gauss(x, *params1)
    gfit2 = gauss(x, *params2)
    
    py.plot(x, y, lw=0.5, label='data')
    py.plot(x,fit,color='red',lw=3,label='gaussian fit')
    py.xlabel('Phase Bin')
    py.ylabel('Intensity')
    py.legend()

    py.plot(x, gfit1, "g")
    py.fill_between(x, gfit1.min(), gfit1, facecolor="green", alpha=0.5)
  
    py.plot(x, gfit2, "y")
    py.fill_between(x, gfit2.min(), gfit2, facecolor="yellow", alpha=0.5)

    return 

In [ ]:
def plot_peaks(x,y,profile_comp):
    py.plot((360/1024)*x,y, lw=0.5)
    py.plot((360/1024)*x, fit, color='red', lw=3, label='gaussian fit')

    x=[profile_comp[i][0] for i in range(1,3)]
    y=[profile_comp[i][1] for i in range(1,3)]

    py.plot(x, y ,'X', markerfacecolor='black', markeredgecolor='black', label='peaks')

    py.xlabel('Phase (deg)')
    py.ylabel('Intensity')
    py.legend()
    return

Functions for Fitting Separation Data

In [ ]:
def separation_singleprofile(components_array, array_wuncert):
    sep_array=[]
    uncert=[]

    for i in range(0,len(components_array)):
        if components_array[i][1][0] != 0 and components_array[i][2][0] != 0:
    
            freq=components_array[i][0]
            sep=(components_array[i][2][0]) - (components_array[i][1][0])

            single_sep=component_separation(frequency=freq, separation=sep)

            sep_array.append(single_sep)

            uncert.append(np.sqrt((array_wuncert[i][3][0])**2 + (array_wuncert[i][3][3])**2))
      
    return sep_array, uncert

In [ ]:
def powerlaw(x, A, alpha, smin):
  return (A * (x**(-alpha)) + smin)

In [ ]:
def powerlawp(x, A, alpha, smin):
  return (A * (x**(alpha)) + smin)

# Importing Data

Data must be in text file format from software pdv.
Also must contain time scrunched data so that the first column of numbers is always 0 (only one pulse). 

In [ ]:
fulldatatype=([('pulse','i8'),('frequency','i8'),('phase','i8'),('intensity','f8')])

In [ ]:
single_pulse = np.loadtxt(fname='../archivefiles/s_archivefiles/f128/pdv/test_pulses/pulse_1503960518.pdv', dtype=fulldatatype)

In [ ]:
no_freq_bands = max(single_pulse['frequency'])+1

# Gaussian Plotting of Individual Frequency Band - Used for error checking individual frequency bands

Choosing Frequency Band. Following Section is for error checking:

In [ ]:
freqband = 11
f = single_pulse['frequency'] == freqband

phase=single_pulse['phase'][f]
intensity=single_pulse['intensity'][f]
x = phase
y = intensity

In [ ]:
len(y)

In [ ]:
py.plot(x,y)

In [ ]:
noise = sigmaClip(y)
error = np.nanstd(noise[1])
print(error)

Gaussian Plots

In [ ]:
fit_number=2
fit, params, sigma, chi=fitting_gaussians(x,y,error, fit_number)

In [ ]:
if fit_number!=0:
    fit, params, sigma, chi=fitting_gaussians(x,y,error,fit_number)
        
if fit!=[]:
    pulse_subp = gauss_peaks(fit, no_freq_bands, freqband, sigma, fit_number)

In [ ]:
if fit!=[]:
    plot_peaks(x,y, pulse_subp)

# Gaussian Plotting and Finding Peaks of Overall Fit Data (fit_components) and Individual Fit Data (ind_components)


The following is a looped version of the previous section to find the correct peaks for each pulse profile

In [ ]:
fit_spulse = []

In [ ]:
fit_number=find_fit(no_freq_bands,single_pulse)

In [ ]:
for i in range(no_freq_bands):
    
    freqband = i
  
    f = single_pulse['frequency'] == freqband

    x=single_pulse['phase'][f]
    y=single_pulse['intensity'][f]
    
    if fit_number!=0:
        fit, params, sigma, chi=fitting_gaussians(x,y,error,fit_number)
        
        if fit!=[]:
            pulse_subp = gauss_peaks(fit, no_freq_bands, freqband, sigma, fit_number)

    
    fit_spulse.append(pulse_subp)
    

In [ ]:
fit_spulse

# Change of Peak Position over Frequency

## Fit Peaks (fit_components)

Starting to graph out all the movement of component phase across frequency

---



In [ ]:
phase_comp1=[]
frequency_comp1=[]

for i in range(0,len(fit_spulse)):
    if fit_spulse[i][1][0] != 0:
        phase_comp1.append(fit_spulse[i][1][0])
        frequency_comp1.append(fit_spulse[i][0])

In [ ]:
py.plot(phase_comp1,frequency_comp1, '.')
py.title('Component 1 - Fit Peaks')
py.xlabel('Phase (deg))')
py.ylabel('Frequency')

In [ ]:
phase_comp2=[]
frequency_comp2=[]

for i in range(0,len(fit_spulse)):
    if fit_spulse[i][2][0] != 0:
        phase_comp2.append(fit_spulse[i][2][0])
        frequency_comp2.append(fit_spulse[i][0])

In [ ]:
py.plot(phase_comp2,frequency_comp2, '.')
py.title('Component 2 - Fit Peaks')
py.xlabel('Phase Bin (deg)')
py.ylabel('Frequency')

## Individual Peaks (ind_components)

In [ ]:
phase_ind1=[]
frequency_ind1=[]

for i in range(0,len(ind_components)):
  if ind_components[i][1][0] != 0:
    phase_ind1.append(ind_components[i][1][0])
    frequency_ind1.append(ind_components[i][0])

In [ ]:
py.plot(phase_ind1,frequency_ind1, '.')
py.title('Component 1 - Individual Peaks')
py.xlabel('Phase Bin (deg)')
py.ylabel('Frequency')

In [ ]:
phase_ind2=[]
frequency_ind2=[]

for i in range(0,len(ind_components)):
  if ind_components[i][2][0] != 0:
    phase_ind2.append(ind_components[i][2][0])
    frequency_ind2.append(ind_components[i][0])

In [ ]:
py.plot(phase_ind2,frequency_ind2, '.')
py.title('Component 2 - Individual Peaks')
py.xlabel('Phase Bin (deg)')
py.ylabel('Frequency')

# Component Separation Across Frequency 

## Fit Peaks (fit_components)

In [ ]:
component_sep, sep_error = separation_singleprofile(fit_components, ind_components)

In [ ]:
comp_sep=[]
frequency_forsep=[]

#range(0,no_freq_bands)
for i in range(0,len(component_sep)):
  comp_sep.append(component_sep[i][1])
  frequency_forsep.append(component_sep[i][0])

In [ ]:
py.plot(frequency_forsep, comp_sep, '.')
py.xlabel('Frequency')
py.ylabel('\u0394\u03B8 (deg)')

## Individual Peaks (ind_components)

In [ ]:
component_sep_ind, ind_sep_errors= separation_singleprofile(ind_components, ind_components)

In [ ]:
comp_sep_ind=[]
frequency_forsep_ind=[]

#range(0,no_freq_bands)
for i in range(0,len(component_sep_ind)):
  comp_sep_ind.append(component_sep_ind[i][1])
  frequency_forsep_ind.append(component_sep_ind[i][0])

In [ ]:
py.plot(frequency_forsep_ind, comp_sep_ind, '.')
py.xlabel('Frequency')
py.ylabel('\u0394\u03B8 (deg)')

# Fitting Power Laws to Separation 

## Fit Peaks (fit_components)

In [ ]:
#To use in weightings: sigma=inv_var
inv_var = []

for i in range(len(sep_error)):
  temp = 1/(sep_error[i]*sep_error[i])
  inv_var.append(temp)

In [ ]:
expect=(703,0.4,16)
sep_params,sep_cov=curve_fit(powerlaw,frequency_forsep,comp_sep, expect, maxfev=10000, sigma=sep_error, bounds=([-np.inf,-np.inf,0],[np.inf,np.inf,500]))
sep_fit_error=np.sqrt(np.diag(sep_cov))

In [ ]:
print(sep_params)
print(sep_fit_error)
separation_fit = powerlaw(frequency_forsep, *sep_params)

In [ ]:
py.plot(frequency_forsep, comp_sep, lw=0.5, label='data')
py.plot(frequency_forsep,separation_fit, color='red',lw=1,label='power fit')
py.xlabel('Frequency')
py.ylabel('\u0394\u03B8 (deg)')

In [ ]:
chi, p = sp.stats.chisquare(comp_sep, separation_fit)
print(f'The equation of fit for this pulsar is: \t \u0394\u03B8 = {sep_params[0]:.3f} \u03BD^-{sep_params[1]:.3f} + {sep_params[2]:.3f} \n')
print(f'The errors are {sep_fit_error[0]:.3f}, {sep_fit_error[1]:.3f}, {sep_fit_error[2]:.3f} ')
print('The chi squared value for the fit is: ', chi)
print('Units for \u0394\u03B8 is degrees and \u03BD is MHz')

## Individual Peaks (ind_components)

In [ ]:
#To use in weightings: sigma=inv_var_ind
inv_var_ind = []

for i in range(len(ind_sep_errors)):
  temp = 1/(ind_sep_errors[i]*ind_sep_errors[i])
  inv_var_ind.append(temp)

In [ ]:
expect=(200,0.5,50)
sep_params_ind,sep_cov_ind=curve_fit(powerlaw,frequency_forsep_ind,comp_sep_ind, expect, maxfev=1000000, sigma=inv_var_ind,bounds=([-np.inf,-np.inf,0],[np.inf,np.inf,500]))
sep_fit_error_ind=np.sqrt(np.diag(sep_cov_ind))

In [ ]:
print(sep_params_ind)
separation_fit_ind = powerlaw(frequency_forsep, *sep_params_ind)

In [ ]:
py.plot(frequency_forsep_ind, comp_sep_ind, lw=0.5, label='data')
py.plot(frequency_forsep_ind,separation_fit_ind, color='red',lw=1,label='power fit')
py.xlabel('Frequency')
py.ylabel('\u0394\u03B8 (deg)')

In [ ]:
chi_ind, p_ind = sp.stats.chisquare(comp_sep, separation_fit)
print(f'The equation of fit for this pulsar is: \t \u0394\u03B8 = {sep_params_ind[0]:.3f} \u03BD^-{sep_params_ind[1]:.3f} + {sep_params_ind[2]:.3f} \n')
print(f'The errors are {sep_fit_error_ind[0]:.3f}, {sep_fit_error_ind[1]:.3f}, {sep_fit_error_ind[2]:.3f} ')
print('The chi squared value for the fit is: ', chi_ind)
print('Units for \u0394\u03B8 is degrees and \u03BD is MHz')

# Change in Intensity over Frequency

## Fit Peaks (fit_components)

In [ ]:
intensity_comp1=[]
frequency_comp1=[]

for i in range(0,len(fit_components)):
  if fit_components[i][1][0] != 0:
    intensity_comp1.append(fit_components[i][1][1])
    frequency_comp1.append(fit_components[i][0])

In [ ]:
py.plot(frequency_comp1,intensity_comp1, '.')
py.title('Component 1 - Fit Peaks')
py.xlabel('Frequency')
py.ylabel('Intensity')

In [ ]:
intensity_comp2=[]
frequency_comp2=[]

for i in range(0,len(fit_components)):
  if fit_components[i][1][0] != 0:
    intensity_comp2.append(fit_components[i][2][1])
    frequency_comp2.append(fit_components[i][0])

In [ ]:
py.plot(frequency_comp2,intensity_comp2, '.')
py.title('Component 2 - Fit Peaks')
py.xlabel('Frequency')
py.ylabel('Intensity')

## Individual Peaks (ind_components)

In [ ]:
intensity_ind1=[]
frequency_ind1=[]

for i in range(0,len(ind_components)):
  if ind_components[i][1][0] != 0:
    intensity_ind1.append(ind_components[i][1][1])
    frequency_ind1.append(ind_components[i][0])

In [ ]:
py.plot(frequency_ind1,intensity_ind1, '.')
py.title('Component 1 - Individual Peaks')
py.xlabel('Frequency')
py.ylabel('Intensity')

In [ ]:
intensity_ind2=[]
frequency_ind2=[]

for i in range(0,len(ind_components)):
  if ind_components[i][1][0] != 0:
    intensity_ind2.append(ind_components[i][2][1])
    frequency_ind2.append(ind_components[i][0])

In [ ]:
py.plot(frequency_ind2,intensity_ind2, '.')
py.title('Component 2 - Individual Peaks')
py.xlabel('Frequency')
py.ylabel('Intensity')

# Intensity Fitting

In [ ]:
int1_params,int1_cov=curve_fit(powerlaw,frequency_ind1,intensity_ind1, maxfev=10000)
int1_error=np.sqrt(np.diag(int1_cov))

In [ ]:
int1_fit = powerlaw(frequency_ind1, *int1_params)

In [ ]:
py.plot(frequency_ind1, intensity_ind1, lw=0.5, label='data')
py.plot(frequency_ind1, int1_fit, color='red',lw=1,label='power fit')
py.xlabel('Frequency')
py.ylabel('\u0394\u03B8 (deg)')

In [ ]:
int2_params,int2_cov=curve_fit(powerlaw,frequency_ind2,intensity_ind2, maxfev=10000)
int2_error=np.sqrt(np.diag(int2_cov))

In [ ]:
int2_fit = powerlaw(frequency_ind2, *int2_params)

In [ ]:
py.plot(frequency_ind2, intensity_ind2, lw=0.5, label='data')
py.plot(frequency_ind2, int1_fit, color='red',lw=1,label='power fit')
py.xlabel('Frequency')
py.ylabel('\u0394\u03B8 (deg)')

# Change in Width over Frequency

In [ ]:
width_ind1=[]
frequency_ind1=[]

for i in range(0,len(ind_components)):
  if ind_components[i][1][0] != 0:
    width_ind1.append(ind_components[i][1][2])
    frequency_ind1.append(ind_components[i][0])

In [ ]:
py.plot(frequency_ind1,width_ind1, '.')
py.title('Component 1 - Individual Peaks')
py.xlabel('Frequency')
py.ylabel('Width (deg)')

In [ ]:
width_ind2=[]
frequency_ind2=[]

for i in range(0,len(ind_components)):
  if ind_components[i][1][0] != 0:
    width_ind2.append(ind_components[i][2][2])
    frequency_ind2.append(ind_components[i][0])

In [ ]:
py.plot(frequency_ind2,width_ind2, '.')
py.title('Component 2 - Individual Peaks')
py.xlabel('Frequency')
py.ylabel('Width (deg)')

# Width Fitting

In [ ]:
wid1_params,wid1_cov=curve_fit(powerlaw,frequency_ind1,width_ind1, maxfev=10000)
wid1_error=np.sqrt(np.diag(wid1_cov))

In [ ]:
wid1_fit = powerlaw(frequency_ind1, *wid1_params)

In [ ]:
py.plot(frequency_ind1, width_ind1, lw=0.5, label='data')
py.plot(frequency_ind1,wid1_fit, color='red',lw=1,label='power fit')
py.xlabel('Frequency')
py.ylabel('\u0394\u03B8 (deg)')

In [ ]:
wid2_params,wid2_cov=curve_fit(powerlaw,frequency_ind2,width_ind2, maxfev=10000)
wid2_error=np.sqrt(np.diag(wid2_cov))

In [ ]:
wid2_fit = powerlaw(frequency_ind2, *wid2_params)

In [ ]:
py.plot(frequency_ind2, width_ind2, lw=0.5, label='data')
py.plot(frequency_ind2, wid2_fit, color='red',lw=1,label='power fit')
py.xlabel('Frequency')
py.ylabel('\u0394\u03B8 (deg)')